trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae

https://translate.yandex.net/api/v1.5/tr/translate
 ? key=<API key>
 & text=<text to translate>
 & lang=<translation direction>
 & [format=<text format>]
 & [options=<translation options>]

In [1]:
from pathlib import Path
import os
import re
from tqdm import tqdm
from pathlib import Path

os.chdir('../')

from src.data import read_sentences

In [2]:
API_KEY = ''

## Reading data

In [3]:
input_sent_semcor, sent_to_id_semcor, target_sent_semcor = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Evaluation_Datasets/', 'ALL', False)

In [4]:
len(input_sent_semcor)

1173

## Translation

In [5]:
id_file = open('data/translation/dev/en_pt/id.txt', 'w')
tr_file = open('data/translation/dev/en_pt/tr.txt', 'w')
or_file = open('data/translation/dev/en_pt/or.txt', 'w')
al_file = open('data/translation/dev/en_pt/al.txt', 'w')

In [13]:
API_KEY = 'trnsl.1.1.20190216T170716Z.034edf2a933741cc.4378585f680c1e47e8bdacd36d2cbdfdd1ec9c2c'

In [15]:
from tqdm import tqdm
import requests

error_api = False
error_reg = False
i = 0
for sentence in tqdm(input_sent_semcor):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                      data={'key': API_KEY,\
                            'text': sentence,\
                            'lang': 'en-pt',\
                            'options': '4'})
    
    if '<Error' in r.text:
        error_api = True
        break
    else:
        tr = re.findall('<text>(.*?)</text>',r.text)
        al = re.findall('<align>(.*?)</align>',r.text)
        if len(tr) and len(al):
            id_file.write(str(i) + '\n')
            tr_file.write(tr[0] + '\n')
            al_file.write(al[0] + '\n')
            or_file.write(sentence + '\n')
        else:
            error_reg = True
            i += 1
    

100%|██████████| 145/145 [02:52<00:00,  1.15s/it]


In [16]:
tr_file.close()
al_file.close()
or_file.close()
id_file.close()

## Backtranslation

In [17]:
tr_file = open('data/translation/dev/en_pt/tr.txt', 'r')


bck_tr_file = open('data/translation/dev/pt_en/tr.txt', 'w')
bck_or_file = open('data/translation/dev/pt_en/or.txt', 'w')
bck_al_file = open('data/translation/dev/pt_en/al.txt', 'w')

In [18]:
input_sent_semcor = []
for line in tr_file:
    input_sent_semcor.append(line.replace('\n', ''))

In [21]:
error_api = False
error_reg = False

for sentence in tqdm(input_sent_semcor):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                      data={'key': API_KEY,\
                            'text': sentence.replace("\n", ""),\
                            'lang': 'pt-en',\
                            'options': '4'})
    
    if '<Error' in r.text:
        error_api = True
        break
    else:
        tr = re.findall('<text>(.*?)</text>',r.text)
        al = re.findall('<align>(.*?)</align>',r.text)
        if len(tr) and len(al):
            bck_tr_file.write(tr[0] + '\n')
            bck_al_file.write(al[0] + '\n')
            bck_or_file.write(sentence + '\n')
            i += 1
        else:
            error_reg = True
            break
            

100%|██████████| 145/145 [02:52<00:00,  1.20s/it]


In [22]:
tr_file.close()
bck_tr_file.close()
bck_al_file.close()
bck_or_file.close()

## Check files

In [23]:
tr_file = open('data/translation/dev/en_pt/dev_tr.txt', 'r')
or_file = open('data/translation/dev/en_pt/dev_or.txt', 'r')
bck_tr_file = open('data/translation/dev/pt_en/dev_tr.txt', 'r')
bck_or_file = open('data/translation/dev/pt_en/dev_or.txt', 'r')

In [24]:
input_or = []
for sent in or_file:
    input_or.append(sent)

input_tr = []
for sent in tr_file:
    input_tr.append(sent)


In [25]:
bck_input_or = []
for sent in bck_or_file:
    bck_input_or.append(sent)

bck_input_tr = []
for sent in bck_tr_file:
    bck_input_tr.append(sent)    

In [26]:
ix= 0
input_or[ix]

'they belong to a group of 15 ringers -- including two octogenarians and four youngsters in training -- who drive every sunday from church to church in a sometimes-exhausting effort to keep the bells sounding in the many belfries of east anglia .\n'

In [27]:
input_tr[ix]

'eles pertencem a um grupo de 15 toques, incluindo dois octogenários e quatro jovens em formação-que unidade de cada domingo da igreja para a igreja, às vezes desgastante esforço para manter os sinos soando em muitos campanários de east anglia .\n'

In [28]:
bck_input_or[ix]

'eles pertencem a um grupo de 15 toques, incluindo dois octogenários e quatro jovens em formação-que unidade de cada domingo da igreja para a igreja, às vezes desgastante esforço para manter os sinos soando em muitos campanários de east anglia .\n'

In [29]:
bck_input_tr[ix]

'they belong to a group of 15 taps, including two octogenarians and four youngsters in training-that drive every Sunday from church to church, sometimes exhausting effort to keep the bells sounding in the many belfries of east anglia .\n'

In [161]:
tr_file.close()
bck_tr_file.close()